In [ ]:
from transformers import Wav2Vec2Config
model_name = "nguyenvulebinh/wav2vec2-base-vietnamese-250h"
config = Wav2Vec2Config.from_pretrained(model_name)

/home/spring/miniconda3/lib/python3.12/site-packages/transformers/configuration_utils.py:306: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
import tvm
from tvm import dlight, relax, te, tir
from tvm.relax import register_pipeline
from tvm.relax.frontend import nn
from tvm.relax.frontend.nn import Tensor, op
from tvm.relax.frontend.nn.llm.kv_cache import PagedKVCache, TIRPagedKVCache
from tvm.runtime import ShapeTuple

## Linear layer

In [ ]:
class Wav2Vec2Linear(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(Wav2Vec2Linear, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias)
    def forward(self, input_values):
        return self.linear(input_values)

In [ ]:
mod, params = Wav2Vec2Linear(768, 768).export_tvm({"forward": {"input_values": nn.spec.Tensor((768,768), "float32")}})

In [ ]:
mod.show()

## Feature Extractor Layer

In [ ]:
# import tvm
# from tvm import dlight, relax, te, tir
# from tvm.relax import register_pipeline
# from tvm.relax.frontend import nn
# from tvm.relax.frontend.nn import Tensor, op
# from tvm.relax.frontend.nn.llm.kv_cache import PagedKVCache, TIRPagedKVCache
# from tvm.runtime import ShapeTuple

class Wav2VecNoLayerNormConvLayer(nn.Module):

    def __init__(self, config, layer_id = 0):
        super().__init__()
        self.in_conv_dim = config.conv_dim[layer_id - 1] if layer_id > 0 else 1
        self.out_conv_dim = config.conv_dim[layer_id]

        self.conv = nn.Conv1D(
            self.in_conv_dim,
            self.out_conv_dim,
            kernel_size=config.conv_kernel[layer_id],
            stride=config.conv_stride[layer_id],
            bias= config.conv_bias,
        )

        self.activation = nn.ReLU()

    def forward(self, hidden_states):
        hidden_states = self.conv(hidden_states)
        hidden_states = self.activation(hidden_states)
        return hidden_states


class Wav2Vec2GroupNormConvLayer(nn.Module):

    def __init__(self, config, layer_id = 0):
        super().__init__()
        self.in_conv_dim = config.conv_dim[layer_id - 1] if layer_id > 0 else 1
        self.out_conv_dim = config.conv_dim[layer_id]

        self.conv = nn.Conv1D(
            self.in_conv_dim,
            self.out_conv_dim,
            kernel_size=config.conv_kernel[layer_id],
            stride=config.conv_stride[layer_id],
            bias= config.conv_bias,
        )

        self.activation = nn.ReLU()
        self.layer_norm = nn.GroupNorm(num_groups=self.out_conv_dim, num_channels=self.out_conv_dim, affine=True)


    def forward(self, hidden_states):
        hidden_states = self.conv(hidden_states)
        hidden_states = self.activation(hidden_states)
        hidden_states =self.layer_norm(hidden_states)
        return hidden_states


class Wav2Vec2FeatureEncoder(nn.Module):

    def __init__(self, config):
        super().__init__()
        conv_layers = [Wav2Vec2GroupNormConvLayer(config, layer_id = 0)] + [Wav2VecNoLayerNormConvLayer(config, layer_id = i+1) for i in range(config.num_feat_extract_layers -1)]
        self.conv_layers = nn.ModuleList(conv_layers)

    def forward(self, input_values):
        hidden_states = input_values
        for conv_layer in self.conv_layers:
            hidden_states = conv_layer(hidden_states)
        return hidden_states

mod, params = Wav2Vec2FeatureEncoder(config).export_tvm({"forward": {"input_values": nn.spec.Tensor((1, 1, 86000), "float32")}})
mod.show()

## Feature Projection layer

In [ ]:
class Wav2Vec2FeatureProjection(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layer_norm = nn.LayerNorm(config.conv_dim[-1], eps=config.layer_norm_eps)
        self.projection = nn.Linear(config.conv_dim[-1], config.hidden_size)
        # self.dropout = nn.Dropout(config.proj_dropout)

    def forward(self, hidden_states):
        hidden_states = self.projection(hidden_states)
        # hidden_states = self.dropout(hidden_states)
        return hidden_states

mod, params = Wav2Vec2FeatureProjection(config).export_tvm({"forward": {"hidden_states": nn.spec.Tensor((1, 268, 512), "float32")}})
mod.show()

## Attention is all you need

In [ ]:
from typing import Optional, Tuple
class Wav2Vec2Attention(nn.Module):
    def __init__(
            self,
            embed_dim: int,
            num_heads: int,
            dropout: float = 0.0,
            is_decoder: bool = False,
            bias: bool = True,
            is_causal: bool = False,
            config: Optional[Wav2Vec2Config] = None,
    ):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.dropout = dropout
        self.head_dim = embed_dim // num_heads
        self.config = config

        if (self.head_dim * num_heads) != embed_dim:
            raise ValueError(
                f"embed_dim must be divisible by num_heads (got `embed_dim`: {embed_dim}, `num_heads`: {num_heads})"
            )

        self.scaling = self.head_dim ** -0.5
        self.is_decoder = is_decoder
        self.is_causal = is_causal

        self.k_proj = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.v_proj = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.q_proj = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.out_proj = nn.Linear(embed_dim, embed_dim, bias=bias)

    def _shape(self, x, seq_len, bsz: int):
        x =  op.reshape(x, (bsz, seq_len, self.num_heads, self.head_dim))
        x = op.permute_dims(x, [0, 2, 1, 3])

        return x

    def forward(
           self,
            hidden_states: Tensor,
            key_value_states: Optional[Tensor] = None,
            past_key_value: Optional[Tuple[Tensor]] = None,
            attention_mask: Optional[Tensor] = None,
            layer_head_mask: Optional[Tensor] = None,
            output_attentions: bool = False,

    )-> Tuple[Tensor, Optional[Tensor], Optional[Tuple]]:
        """Input shape: Batch x Time x Channel"""
        is_cross_attention = key_value_states is not None

        bsz, tgt_len, _ = hidden_states.shape
        query_states = self.q_proj(hidden_states) * self.scaling
        # get key, value proj
        # `past_key_value[0].shape[2] == key_value_states.shape[1]`
        # is checking that the `sequence_length` of the `past_key_value` is the same as
        # the provided `key_value_states` to support prefix tuning
        if (
            is_cross_attention
            and past_key_value is not None
            and past_key_value[0].shape[2] == key_value_states.shape[1]
        ):
            # reuse k,v, cross_attentions
            key_states = past_key_value[0]
            value_states = past_key_value[1]
        elif is_cross_attention:
            # cross_attentions
            key_states = self._shape(self.k_proj(key_value_states), -1, bsz)
            value_states = self._shape(self.v_proj(key_value_states), -1, bsz)
        elif past_key_value is not None:
            # reuse k, v, self_attention
            key_states = self._shape(self.k_proj(hidden_states), -1, bsz)
            value_states = self._shape(self.v_proj(hidden_states), -1, bsz)
            # key_states = cat([past_key_value[0], key_states], dim=2)
            # value_states = cat([past_key_value[1], value_states], dim=2)
            key_states = op.concatenate([past_key_value[0], key_states], axis=2)
            value_states = op.concatenate([past_key_value[1], value_states], axis=2)

        else:
            # self_attention
            key_states = self._shape(self.k_proj(hidden_states), -1, bsz)
            value_states = self._shape(self.v_proj(hidden_states), -1, bsz)

        if self.is_decoder:
            # if cross_attention save Tuple(torch.Tensor, torch.Tensor) of all cross attention key/value_states.
            # Further calls to cross_attention layer can then reuse all cross-attention
            # key/value_states (first "if" case)
            # if uni-directional self-attention (decoder) save Tuple(torch.Tensor, torch.Tensor) of
            # all previous decoder key/value_states. Further calls to uni-directional self-attention
            # can concat previous decoder key/value_states to current projected key/value_states (third "elif" case)
            # if encoder bi-directional self-attention `past_key_value` is always `None`
            past_key_value = (key_states, value_states)

        proj_shape = (bsz * self.num_heads, -1, self.head_dim)
        query_states = self._shape(query_states, tgt_len, bsz).reshape(*proj_shape)
        key_states = key_states.reshape(*proj_shape)
        value_states = value_states.reshape(*proj_shape)
        print(f"key_states shape: {key_states.shape}")
        src_len = key_states.shape[1]

        # attn_weights = torch.bmm(query_states, key_states.transpose(1, 2))
        attn_weights = op.matmul(query_states, op.permute_dims(key_states, [0, 2, 1]))
        print(type(attn_weights))
        if attn_weights.shape != (bsz * self.num_heads, tgt_len, src_len):
            # raise ValueError(
            #     f"Attention weights should be of shape {(bsz * self.num_heads, tgt_len, src_len)}, but is"
            #     f" {attn_weights.shape}"
            # )
            pass

        if attention_mask is not None:
            if attention_mask.shape() != (bsz, 1, tgt_len, src_len):
                raise ValueError(
                    f"Attention mask should be of shape {(bsz, 1, tgt_len, src_len)}, but is {attention_mask.shape()}"
                )
            attn_weights = attn_weights.reshape(bsz, self.num_heads, tgt_len, src_len) + attention_mask
            attn_weights = attn_weights.reshape(bsz * self.num_heads, tgt_len, src_len)

        attn_weights = nn.softmax(attn_weights,axis=-1)

        if layer_head_mask is not None:
            if layer_head_mask.shape() != (self.num_heads,):
                raise ValueError(
                    f"Head mask for a single layer should be of shape {(self.num_heads,)}, but is"
                    f" {layer_head_mask.shape()}"
                )
            attn_weights = layer_head_mask.reshape(1, -1, 1, 1) * attn_weights.reshape(bsz, self.num_heads, tgt_len, src_len)
            attn_weights = attn_weights.reshape(bsz * self.num_heads, tgt_len, src_len)

        if output_attentions:
            # this operation is a bit awkward, but it's required to
            # make sure that attn_weights keeps its gradient.
            # In order to do so, attn_weights have to be reshaped
            # twice and have to be reused in the following
            attn_weights_reshaped = attn_weights.reshape(bsz, self.num_heads, tgt_len, src_len)
            attn_weights = attn_weights_reshaped.reshape(bsz * self.num_heads, tgt_len, src_len)
        else:
            attn_weights_reshaped = None

        # attn_probs = nn.functional.dropout(attn_weights, p=self.dropout, training=self.training)
        attn_probs = attn_weights

        # attn_output = torch.bmm(attn_probs, value_states)
        attn_output = op.matmul(attn_probs, value_states)

        # if attn_output.shape() != (bsz * self.num_heads, tgt_len, self.head_dim):
        #     raise ValueError(
        #         f"`attn_output` should be of shape {(bsz * self.num_heads, tgt_len, self.head_dim)}, but is"
        #         f" {attn_output.shape()}"
        #     )

        # attn_output = attn_output.view(bsz, self.num_heads, tgt_len, self.head_dim)
        # attn_output = attn_output.transpose(1, 2)
        attn_output = op.reshape(attn_output, (bsz, self.num_heads, tgt_len, self.head_dim))
        # attn_output = op.transpose(attn_output, (0, 2, 1, 3))
        attn_output = op.permute_dims(attn_output, [0, 2, 1, 3])

        # Use the `embed_dim` from the config (stored in the class) rather than `hidden_state` because `attn_output` can be
        # partitioned across GPUs when using tensor-parallelism.
        attn_output = attn_output.reshape(bsz, tgt_len, self.embed_dim)

        attn_output = self.out_proj(attn_output)

        return attn_output, attn_weights_reshaped, past_key_value


In [ ]:
mod, params = Wav2Vec2Attention(512, 8).export_tvm({"forward": {"hidden_states": nn.spec.Tensor((1, 268, 512), "float32")}})
mod.show()

key_states shape: [8, 268, 64]
<class 'tvm.relax.frontend.nn.core.Tensor'>


[14:55:50] /home/spring/tvm/src/relax/ir/block_builder.cc:65: Warning: BlockBuilder destroyed with remaining blocks!


TypeError: Unsupported return type: <class 'NoneType'>

## Encoder layer

In [ ]:
class Wav2Vec2EncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.attention = nn.MultiHeadAttention(
            embed_dim=config.hidden_size,
            num_heads=config.num_attention_heads,
            dropout=config.attention_dropout,
        )
        self.layer_norm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.ffn = nn.Sequential(
            nn.Linear(config.hidden_size, config.intermediate_size),
            nn.GELU(),
            nn.Linear(config.intermediate_size, config.hidden_size),
        )
        self.dropout = nn.Dropout(config.hidden_dropout)

## Compiling to DPU

In [ ]:
import pyxir
import pyxir.contrib.target.DPUCADF8H

import tvm
import tvm.relay as relay
from tvm.contrib.target import vitis_ai
from tvm.contrib import utils, graph_executor
from tvm.relay.op.contrib.vitis_ai import partition_for_vitis_ai

tvm_target = 'llvm'
dpu_target = 'DPUCZDX8G-kv260' # options: 'DPUCADF8H', 'DPUCAHX8H-u50', 'DPUCAHX8H-u280', 'DPUCAHX8L', 'DPUCVDX8H', 'DPUCZDX8G-zcu104', 'DPUCZDX8G-zcu102', 'DPUCZDX8G-kv260'